# Subset-Sum Problem (SSP)

The **Subset Sum problem (SSP)** is defined as follows:

>Given a set $S = \{x_1, x_2, \ldots, x_n\}$ of positive integers,
>and a positive integers $t$, is there a subset of $S$ whose sum is equal to $t$?

This is the **decision version** of SSP as it only asks for a true/false answer.

#### Example

$S=\{1,2,3,10\}$ and $t=13$.

A solution is given by $\{1,2,10\}$. There is also another solution: $\{3,10\}$.

## Exhaustive search

An **exhaustive search algorithm** for this problem can written *iteratively* as follows:

**INPUT:** A set $S = \{x_1, x_2, \ldots, x_n\}$ of positive integers, and a positive integer $t$.

**OUTPUT:** $(c_1,\ldots,c_n)$ such that $\sum_{i=1}^n c_i x_i = t$

1. **for all** $(c_1,\ldots,c_n)\in\{0,1\}^n$ **do**
2. $\quad$ **if** $\sum_{i=1}^n c_i x_i = t$ **then**
3. $\qquad$ **return** $(c_1,\ldots,c_n)$

Now think of the exhaustive search as a binary tree, where at each level we decide whether to include the $i^\text{th}$ number or not. For example, if $S = \{a, b, c, d\}$ then we get:

![](img/ssp-binary-tree.jpg)

Write a recursive version of the exhaustive search algorithm.

## Recursive Exhaustive Search for Subset Sum Problem (SSP)

The recursive approach explores all inclusion/exclusion possibilities for each element in the set. At each recursive step, we either include the current element in the subset or exclude it, forming a binary decision tree.


<pre>
def subset_sum(S, t, index=0, current_sum=0, subset=[]):
    # Base case: solution found
    if current_sum == t:
        return subset
    
    # Base case: no solution in this branch
    if index >= len(S) or current_sum > t:
        return None
    
    # Recursive case 1: exclude current element
    result = subset_sum(S, t, index+1, current_sum, subset.copy())
    if result is not None:
        return result
    
    # Recursive case 2: include current element
    subset.append(S[index])
    result = subset_sum(S, t, index+1, current_sum + S[index], subset.copy())
    if result is not None:
        return result
    
    # Backtrack
    subset.pop()
    return None
</pre>

<p>This recursive algorithm:
<ol>
  <li>Maintains current state (index, sum, subset)</li>
  <li>At each step, explores both inclusion/exclusion of current element</li>
  <li>Returns first valid subset found</li>
  <li>Uses backtracking to explore all possibilities</li>
  <li>Has O(2^n) time complexity</li>
</ol>
</p>

## Dynamic Programming.

The above iterative pseudocode considers the subsets of $S$ as the search space.
Another way to search for solutions is to iteratively build the answer for smaller target values $t' = 0, 1, 2, 3, \ldots$ until we reach $t$.

If we have built all the possible sums from the subset $\{x_1,\ldots,x_k\}$, what other sums become possible if we add $x_{k+1}$ to the set?

<h3>Dynamic Programming Insight</h3>
<p>When adding element x<sub>k+1</sub> to existing sums P<sub>k</sub>:</p>
<ol>
  <li>All previous sums remain valid (not using x<sub>k+1</sub>)</li>
  <li>New sums are created by adding x<sub>k+1</sub> to each sum in P<sub>k</sub></li>
  <li>The complete set becomes P<sub>k+1</sub> = P<sub>k</sub> ∪ {s+x<sub>k+1</sub> | s ∈ P<sub>k</sub>}</li>
</ol>

<p>Key observations:</p>
<ul>
  <li>Each element potentially doubles the number of possible sums</li>
  <li>The maximum possible sum is sum(S)</li>
  <li>We can represent achievable sums with a boolean array</li>
  <li>This grows the solution space incrementally</li>
</ul>

# How Adding an Element Affects Possible Sums in Subset Sum

Suppose we have a set `S_k = {x1, x2, ..., xk}` and have already computed all achievable subset sums using subsets of `S_k`, stored in the set `P_k`. When a new element `x_{k+1}` is added, we determine the new possible sums.

The updated set of achievable sums, `P_{k+1}`, includes:

1. All sums already in `P_k` (i.e., subsets that **do not** include `x_{k+1}`),
2. All new sums formed by adding `x_{k+1}` to each element of `P_k` (i.e., subsets that **do** include `x_{k+1}`).

**Formally:**

```
P_{k+1} = P_k ∪ { s + x_{k+1} | s ∈ P_k }
```

This idea enables us to **iteratively** construct all possible subset sums starting from an empty set. We begin with:

```
P_0 = {0}
```

since a sum of `0` is always achievable by choosing the empty subset.

At each step `i`, we compute `P_i` from `P_{i-1}` by:

- Copying all existing sums, and  
- Adding `x_i` to each existing sum to get new ones.

This forms the foundation of the **bottom-up dynamic programming approach** to the subset sum problem. It avoids recomputation and ensures each combination is only considered once, making it more efficient than naive recursion, especially when dealing with large sets or targets.


Now explain how we can use this for a bottom-up approach to decide SSP.

<h3>Bottom-Up DP Approach</h3>
<pre>
def dp_subset_sum(S, t):
    dp = [False]*(t+1)
    dp[0] = True  # empty subset
    
    for num in S:
        for j in range(t, num-1, -1):
            if dp[j - num]:
                dp[j] = True
    
    return dp[t]
</pre>

<p>Steps:
<ol>
  <li>Initialize dp array where dp[i] = True if sum i is possible</li>
  <li>Start with dp[0] = True (empty subset)</li>
  <li>For each number:
    <ul>
      <li>Update dp array backwards</li>
      <li>If (j - num) was possible, then j becomes possible</li>
    </ul>
  </li>
  <li>Final answer is dp[t]</li>
</ol>
</p>

<p>Complexity: O(n*t) time, O(t) space</p>
<p>Advantages over recursion:
<ul>
  <li>Polynomial time when t is small</li>
  <li>No recursion stack overflow</li>
  <li>Can reconstruct solution by tracking choices</li>
</ul>
</p>

# Implementation

In [6]:
from random import randint, sample
from typing import List, Optional, Dict, Tuple

In [7]:
def get_S_t(n, MAX_X = 100):
    S = [randint(1,MAX_X) for i in range(n)]
    t = sum(sample(S,randint(1,n)))
    return S,t

In [8]:
get_S_t(10)

([11, 22, 33, 82, 24, 28, 37, 27, 5, 62], 32)

### 1) Memoization (Top-down approach)

In [14]:
#Implementation
def subset_sum_memo(S: List[int], t: int) -> Optional[List[int]]:
    """Top-down DP with memoization for SSP. Returns a subset if found, else None."""
    n = len(S)
    memo: Dict[Tuple[int, int], Optional[List[int]]] = {}

    def backtrack(index: int, remaining: int) -> Optional[List[int]]:
        # Base Case 1: Subset found
        if remaining == 0:
            return []
        # Base Case 2: No solution (exceeded remaining or end of list)
        if remaining < 0 or index == n:
            return None
        # Check memo
        if (index, remaining) in memo:
            return memo[(index, remaining)]

        # Option 1: Exclude S[index]
        result = backtrack(index + 1, remaining)
        if result is not None:
            memo[(index, remaining)] = result
            return result

        # Option 2: Include S[index]
        result = backtrack(index + 1, remaining - S[index])
        if result is not None:
            memo[(index, remaining)] = [S[index]] + result
            return [S[index]] + result

        memo[(index, remaining)] = None
        return None

    return backtrack(0, t)

In [15]:
# Test examples
def test_memoization():
    print("=== Testing Memoization (Top-down) ===")
    # Example from problem statement
    S1 = [1, 2, 3, 10]
    t1 = 13
    print(f"S = {S1}, t = {t1}")
    print("Solution:", subset_sum_memo(S1, t1))  # Expected: [10, 3] or [10, 2, 1]

    # Random test case
    S2, t2 = get_S_t(10)
    print(f"\nRandom Test: S = {S2}, t = {t2}")
    print("Solution:", subset_sum_memo(S2, t2))
test_memoization()

=== Testing Memoization (Top-down) ===
S = [1, 2, 3, 10], t = 13
Solution: [3, 10]

Random Test: S = [49, 87, 2, 22, 98, 50, 6, 25, 48, 61], t = 232
Solution: [98, 25, 48, 61]


### 2) Bottom-up approach

In [16]:
# Implementation
def subset_sum_bottom_up(S: List[int], t: int) -> Optional[List[int]]:
    """Bottom-up DP for SSP. Returns a subset if found, else None."""
    n = len(S)
    # dp[i][j] = True if sum j can be formed with first i elements
    dp = [[False] * (t + 1) for _ in range(n + 1)]
    dp[0][0] = True  # Base case: empty subset sums to 0

    # Fill DP table
    for i in range(1, n + 1):
        for j in range(t + 1):
            if j < S[i - 1]:
                dp[i][j] = dp[i - 1][j]
            else:
                dp[i][j] = dp[i - 1][j] or dp[i - 1][j - S[i - 1]]

    # Early exit if no solution
    if not dp[n][t]:
        return None

    # Backtrack to find the subset
    subset = []
    j = t
    for i in range(n, 0, -1):
        if not dp[i - 1][j]:  # Current element was included
            subset.append(S[i - 1])
            j -= S[i - 1]

    return subset

In [13]:
# Test examples
def test_bottom_up():
    print("\n=== Testing Bottom-up DP ===")
    # Example from problem statement
    S1 = [1, 2, 3, 10]
    t1 = 13
    print(f"S = {S1}, t = {t1}")
    print("Solution:", subset_sum_bottom_up(S1, t1))  # Expected: [10, 3] or [10, 2, 1]

    # Edge case: t = 0
    print(f"\nEdge Case: S = {S1}, t = 0")
    print("Solution:", subset_sum_bottom_up(S1, 0))  # Expected: []

    # Random test case
    S2, t2 = get_S_t(15)
    print(f"\nRandom Test: S = {S2}, t = {t2}")
    print("Solution:", subset_sum_bottom_up(S2, t2))
test_bottom_up()


=== Testing Bottom-up DP ===
S = [1, 2, 3, 10], t = 13
Solution: [10, 2, 1]

Edge Case: S = [1, 2, 3, 10], t = 0
Solution: []

Random Test: S = [20, 13, 32, 94, 97, 71, 62, 36, 13, 75, 67, 54, 42, 60, 40], t = 776
Solution: [40, 60, 42, 54, 67, 75, 13, 36, 62, 71, 97, 94, 32, 13, 20]


# Conclusion

<h4>Summary of Approaches</h4>
<p>The Subset Sum Problem demonstrates how different algorithmic techniques can solve the same problem with varying efficiency. We examined three approaches:</p>
<ol>
  <li><strong>Exhaustive Search</strong> - A recursive solution that explores all possible subsets (O(2<sup>n</sup>)</li>
  <li><strong>Top-Down DP</strong> - Memoization-based approach that avoids redundant computations</li>
  <li><strong>Bottom-Up DP</strong> - Iterative solution that builds solutions from smaller subproblems (O(n*t))</li>
</ol>

<h4>Key Insights</h4>
<ul>
  <li>The DP approaches significantly outperform brute-force for moderate target values</li>
  <li>Bottom-up DP is generally more space-efficient than top-down</li>
  <li>The problem exhibits the classic time-space tradeoff in algorithm design</li>
  <li>Practical implementations often use bitmask optimizations for the DP solution</li>
</ul>

<h4>Applications</h4>
<p>The SSP has real-world applications in:</p>
<ul>
  <li>Financial portfolio optimization</li>
  <li>Resource allocation problems</li>
  <li>Cryptography and knapsack problems</li>
  <li>Decision-making systems in AI</li>
</ul>

# List of references


<ol>
  <li>Kochenderfer, M. J., Wheeler, T. A., & Wray, K. H. (2022). Algorithms for decision making. MIT press. <a> https://mitp-content-server.mit.edu/books/content/sectbyfn/books_pres_0/14348/onesheet.pdf</a></li>
  
  <li>Chen, L., Lian, J., Mao, Y., & Zhang, G. (2024, October). An improved pseudopolynomial time algorithm for subset sum. In 2024 IEEE 65th Annual Symposium on Foundations of Computer Science (FOCS) (pp. 2202-2216). IEEE.<a> https://arxiv.org/pdf/2402.14493</a></li>
  
  <li>Chen, L., Lian, J., Mao, Y., & Zhang, G. (2024). Faster algorithms for bounded knapsack and bounded subset sum via fine-grained proximity results. In Proceedings of the 2024 Annual ACM-SIAM Symposium on Discrete Algorithms (SODA) (pp. 4828-4848). Society for Industrial and Applied Mathematics.<a> https://epubs.siam.org/doi/pdf/10.1137/1.9781611977912.171</a></li>
</ol>